In [ ]:
import pandas as pd

In [ ]:
train = pd.read_csv('../input/train.csv.zip')
test = pd.read_csv('../input/test.csv.zip')

In [3]:
import requests
import re
import time
from random import randint

START_SPELL_CHECK="<span class=\"spell\">Showing results for</span>"
END_SPELL_CHECK="<br><span class=\"spell_orig\">Search instead for"

HTML_Codes = (
        ("'", '&#39;'),
        ('"', '&quot;'),
        ('>', '&gt;'),
        ('<', '&lt;'),
        ('&', '&amp;'),
)

def spell_check(s):
    q = '+'.join(s.split())
    time.sleep(  randint(0,2) ) #relax and don't let google be angry
    r = requests.get("https://www.google.co.uk/search?q="+q)
    content = r.text
    start=content.find(START_SPELL_CHECK) 
    if ( start > -1 ):
        start = start + len(START_SPELL_CHECK)
        end=content.find(END_SPELL_CHECK)
        search= content[start:end]
        search = re.sub(r'<[^>]+>', '', search)
        for code in HTML_Codes:
            search = search.replace(code[1], code[0])
        search = search[1:]
    else:
        search = s
    return search

In [4]:
searches = ["metal plate cover gcfi"]
 
for search in searches:
    speel_check_search= spell_check(search)
    print (search+" -> " + speel_check_search)

metal plate cover gcfi -> metal plate cover gfci


In [5]:
train.columns

Index(['id', 'comment_text', 'toxic', 'severe_toxic', 'obscene', 'threat',
       'insult', 'identity_hate'],
      dtype='object')

In [6]:
from collections import Counter
from tqdm import tqdm
from stemming.porter2 import stem
import re

In [22]:
def tokenizer(s):
    s = re.sub(r"(\w)\.([A-Z])", r"\1 \2", s) #Split words with a.A
    s = re.sub(r"\s([a-z]+)([A-Z])", r" \1 \2", s)
    s = s.replace("$"," ")
    s = s.replace("?"," ")
    s = s.replace("!"," ")
    s = s.replace("-"," ")
    s = s.replace("//","/")
    s = s.replace("..",".")
    s = s.replace(" / "," ")
    s = s.replace(" \\ "," ")
    s = s.replace("("," ")
    s = s.replace(")"," ")
    s = s.replace("{"," ")
    s = s.replace("}"," ")
    s = s.replace(":"," ")
    s = s.replace(".","  ")
    s = s.replace("…","  ")
    s = s.replace(",","  ")
    s = s.replace("''", "  ")
    
    s = s.replace('"',"")
    s = s.replace("'","")
    s = s.lower()
    return s

In [23]:
counter = Counter()
cnt = 0; pool = []
for s in tqdm(train.comment_text.map(tokenizer)):
    cnt +=1
    counter += Counter(s.split())
    if cnt % 10000 == 0 and cnt>0:
        pool.append(counter)
        counter = Counter()
pool.append(counter)

100%|██████████| 159571/159571 [02:58<00:00, 892.67it/s]


In [24]:
counter = Counter()
for p in pool:
    counter += p

In [25]:
counter.most_common()[:9]

[('the', 496113),
 ('to', 297286),
 ('of', 224451),
 ('and', 223248),
 ('a', 215882),
 ('you', 205905),
 ('i', 204242),
 ('is', 176196),
 ('that', 154405)]

In [26]:
df = pd.DataFrame(counter.most_common(), columns=['word', 'freq'])
df.head()

,word,freq
0,the,496113
1,to,297286
2,of,224451
3,and,223248
4,a,215882


In [27]:
df.to_pickle('../data/word_count_train.p')

# test

In [28]:
counter = Counter()
cnt = 0; pool = []
for s in tqdm(test.comment_text.map(tokenizer)):
    cnt +=1
    counter += Counter(s.split())
    if cnt % 10000 == 0 and cnt>0:
        pool.append(counter)
        counter = Counter()
pool.append(counter)

counter = Counter()
for p in pool:
    counter += p

100%|██████████| 153164/153164 [03:10<00:00, 805.90it/s]


In [29]:
df = pd.DataFrame(counter.most_common(), columns=['word', 'freq'])
df.head()

,word,freq
0,the,419546
1,to,241360
2,a,193261
3,of,185287
4,and,183942


In [30]:
df.to_pickle('../data/word_count_test.p')

In [31]:
df

,word,freq
0,the,419546
1,to,241360
2,a,193261
3,of,185287
4,and,183942
5,you,164770
6,i,159731
7,is,157227
8,that,128198
9,in,123444


In [32]:
train.sample(8).comment_text.values

array([ '"\n\n OBE? \n\nHe is listed in the New Year\'s Honours List of 1953 as being awarded the OBE (Military division). Where and when did he serve?  "',
       "Wow...funny how i can harass other users by calling them cocksuckers on my own talk page.\n\nThe concept of Wikipedia is moronic...the general public is stupid, and should not be given complete unfettered access to change globally accessible content.\n\nDumb fucks, I'm sure it completely gets your little shriveled rocks off to fling your term 'vandalism' around like it is worth anything. How about you sit on my huge cock you egotistical power hungry nerd shit eating fagots D\n\nHow is it abuse you fucking idiots when all you have to do is ban me? Stupid fucking idiot dipshits.",
       'bastards, i said nothing about changing it! that was not a complaint!',
       "Frank Lloyd wright \n\nhi all, first of let me state i like Frank Lloyd Wright's work, the 'water fall' hoose is a favourite, but what is it with the near consta

In [33]:
train.sample(9)

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
112500,59d0f1251b462acc,Vandalism \n\nYour vandalism on the article 20...,0,0,0,0,0,0
121892,8c17babdaa3d0dff,"Mother of Fraternities page\nThank you, User 7...",0,0,0,0,0,0
22680,3be78d3e344d7db1,Please stop. If you continue to vandalize Wiki...,0,0,0,0,0,0
55079,932e69679356a660,Deletion of Finbar Prunty\n\nYour logic for no...,0,0,0,0,0,0
109584,4a30c91ebb06f7db,Go to the F1 2016 page. They almost immediatel...,0,0,0,0,0,0
155054,b8324615a1bcaf58,scum who look up to Che deserve the worst kind...,1,0,1,1,1,0
21433,387ae6da41484dd9,Insertion of false information into articles\n...,0,0,0,0,0,0
3991,0aa716e5156bdafe,"""\n\n A barnstar for you! \n\n The Original B...",0,0,0,0,0,0
15761,299f016b8f07252e,The endorsements listed for Peggy Neely on thi...,0,0,0,0,0,0


In [34]:
toxic1 = train[train.toxic==1]
toxic0 = train[train.toxic==0]

In [46]:
toxic1.sample(9).values

array([['0489a254fed03fab',
        'stop leaving stupid harping messages for peopel that are nil to do with what you are talking about',
        1, 0, 0, 0, 0, 0],
       ['c360efc517990f93',
        'You fucking idiot \n\nI edited that page for a reason, the page was terrible it looked some little kid wrote the damn page.',
        1, 0, 1, 0, 1, 0],
       ['b8c57d4e584715d4',
        "He likely thinks that you're an idiot, which you are.", 1, 0, 1,
        0, 1, 0],
       ['e64c55042b42c496',
        '"As much as you can stand? What the fuck? You\'re the one who started harassing me, you have been doing so for monthsfor fuck\'s sake, you showed up at my talkpage in August to ask me a question, which I responded to extremely politely, only to have you throw it in my face. You showed up at ANI to harass me when I had said absofuckinglutely nothing to you, and indeed had said nothing about civility, but had merely pointed out that a given comment was unconstructive. I did not notify 

In [38]:
toxic0.sample(9).values

array([['f0dd37e179dd2999',
        '"\nYou don\'t get it. You\'re too involved with the politics, as exemplified by the detail you\'ve gone into even in that response. It\'s an absolute simple and clear fact that, to most English speakers on Earth, ""Republic of China"" IS synonymous with ""Taiwan"". It\'s not a POV. It\'s the way the rest of the world looks at the reality in that part of the world. People seeing things that way are not making a choice about what POV they should have. To them, it\'s what things are. Yes, the article can go into some detail about the history, but I can assure you that it won\'t change that ""common name"" understanding.   \n\n"',
        0, 0, 0, 0, 0, 0],
       ['4d06786faa79a493',
        '"\n\n Congratulations from STiki! \n\n   Congratulations, Onel5969! You\'re receiving this barnstar of merit because you recently crossed the  classification threshold using STiki.\n\nWe thank you both for your contributions to Wikipedia at-large and your use of t

In [40]:
from nltk.corpus import stopwords

In [44]:
stopWords = set(stopwords.words('english'))

In [47]:
train[train.comment_text.str.contains('yyyyyyy')]

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
8136,15a6722083249ddd,"""\n\nOh, that is RICH indeed!!! """"stay cool""""...",0,0,0,0,0,0
27951,49fa0c9645b5b7a0,"YAMLA, are you stupid and a gay ass fag whos ...",1,0,1,0,1,1
38125,65c35364ff05b38c,Yayyyyyyyyyyyyy Kim just got out of 00j0ail.,0,0,0,0,0,0
69895,bafba36ae5eefb5a,Heeeeeeeeeeeeeeyyyyyyyyyyyy dude! Sup!,0,0,0,0,0,0
72630,c253e034605ee865,is true. so i want to express that grant is ga...,1,0,0,0,0,1
104679,2ffbf0a2b9bd0819,HHHHHHHHHHHHHHAAAAAAHAHA you're funny.. N...,1,0,0,0,0,0
114237,631e6be4596f2731,"I am not making personal attacks, I am comment...",0,0,0,0,0,0
115256,6850c0db302991ee,Kisssssssss me\nHi babyyyyyyyyyyyyyyyyy\ncalll...,0,0,0,0,0,0
115606,6a4a92c6631e7b90,Feel the might of Cyber Apocalypse! \n\nyyyyyy...,0,0,0,0,0,0
151556,7f0f1a7286ecbc51,hayyyyyyyyy \n\nyeah this karoolzzz hay k hayy...,0,0,0,0,0,0


In [49]:
train.iloc[115606].values

array(['6a4a92c6631e7b90',
       'Feel the might of Cyber Apocalypse! \n\nyyyyyyyyyyyyyyybvgtfrrrrrbyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyyy